# **Labs 1 and 2 PySpark:**

In these labs we will be using the "[[NeurIPS 2020] Data Science for COVID-19 (DS4C)](https://www.kaggle.com/datasets/kimjihoo/coronavirusdataset?select=PatientInfo.csv)" dataset, retrieved from [Kaggle](https://www.kaggle.com/) on 1/6/2022, for educational non commercial purpose, License
[CC BY-NC-SA 4.0
](https://creativecommons.org/licenses/by-nc-sa/4.0/)


The csv file that we will be using in this lab is **PatientInfo**.

## PatientInfo.csv

**patient_id**
the ID of the patient

**sex**
the sex of the patient

**age**
the age of the patient

**country**
the country of the patient

**province**
the province of the patient

**city**
the city of the patient

**infection_case**
the case of infection

**infected_by**
the ID of who infected the patient


**contact_number**
the number of contacts with people

**symptom_onset_date**
the date of symptom onset

**confirmed_date**
the date of being confirmed

**released_date**
the date of being released

**deceased_date**
the date of being deceased

**state**
isolated / released / deceased

### Import the pyspark and check it's version

In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=b50f70ea3172f6a7f824054e8b9afde208362593124c4b98f2637fae466cd5da
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
import pyspark
print(pyspark.__version__)

3.5.1


### Import and create SparkSession

In [22]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType

# import libraries

In [11]:
from pyspark.sql import functions as F

### Load the PatientInfo.csv file and show the first 5 rows

In [4]:
# Create Spark session
spark = SparkSession.builder .appName("Patient Analysis") .getOrCreate()

In [5]:
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

### Display the schema of the dataset

In [6]:
df = spark.read.csv('/content/PatientInfo.csv', header=True, inferSchema=True)

In [7]:
df.show(5)

+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|patient_id|   sex|age|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|
+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul| Gangseo-gu|     overseas inflow|       NULL|            75|        2020-01-22|    2020-01-23|   2020-02-05|         NULL|released|
|1000000002|  male|30s|  Korea|   Seoul|Jungnang-gu|     overseas inflow|       NULL|            31|              NULL|    2020-01-30|   2020-03-02|         NULL|released|
|1000000003|  male|50s|  Korea|   Seoul|  Jongno-gu|contact with patient| 2002000001|            17|              NULL|    2020-01-30|   202

### Display the statistical summary

In [8]:
df.printSchema()

root
 |-- patient_id: long (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- country: string (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- infected_by: string (nullable = true)
 |-- contact_number: string (nullable = true)
 |-- symptom_onset_date: string (nullable = true)
 |-- confirmed_date: date (nullable = true)
 |-- released_date: date (nullable = true)
 |-- deceased_date: date (nullable = true)
 |-- state: string (nullable = true)



### Using the state column.
### How many people survived (released), and how many didn't survive (isolated/deceased)?

In [9]:
df.groupBy('state').count().show()

+--------+-----+
|   state|count|
+--------+-----+
|isolated| 2158|
|released| 2929|
|deceased|   78|
+--------+-----+



### Display the number of null values in each column

In [14]:
null_counts = df.select([F.sum(F.col(c).isNull().cast("int")).alias(c) for c in df.columns])
null_counts.show()

+----------+----+----+-------+--------+----+--------------+-----------+--------------+------------------+--------------+-------------+-------------+-----+
|patient_id| sex| age|country|province|city|infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|state|
+----------+----+----+-------+--------+----+--------------+-----------+--------------+------------------+--------------+-------------+-------------+-----+
|         0|1122|1380|      0|       0|  94|           919|       3819|          4374|              4475|             3|         3578|         5099|    0|
+----------+----+----+-------+--------+----+--------------+-----------+--------------+------------------+--------------+-------------+-------------+-----+



## Data preprocessing

### Fill the nulls in the deceased_date with the released_date.
- You can use <b>coalesce</b> function

In [15]:
df = df.withColumn('deceased_date', F.coalesce(df['deceased_date'], df['released_date']))

### Add a column named no_days which is difference between the deceased_date and the confirmed_date then show the top 5 rows. Print the schema.
- <b> Hint: You need to typecast these columns as date first <b>

In [16]:
df = df.withColumn('confirmed_date', F.to_date(df['confirmed_date']))
df = df.withColumn('deceased_date', F.to_date(df['deceased_date']))

In [17]:
df = df.withColumn('no_days', F.datediff(df['deceased_date'], df['confirmed_date']))

In [18]:
df.show(5)

+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+-------+
|patient_id|   sex|age|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|no_days|
+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+-------+
|1000000001|  male|50s|  Korea|   Seoul| Gangseo-gu|     overseas inflow|       NULL|            75|        2020-01-22|    2020-01-23|   2020-02-05|   2020-02-05|released|     13|
|1000000002|  male|30s|  Korea|   Seoul|Jungnang-gu|     overseas inflow|       NULL|            31|              NULL|    2020-01-30|   2020-03-02|   2020-03-02|released|     32|
|1000000003|  male|50s|  Korea|   Seoul|  Jongno-gu|contact with patient| 2002000001|            17|

In [19]:
df.printSchema()

root
 |-- patient_id: long (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- country: string (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- infected_by: string (nullable = true)
 |-- contact_number: string (nullable = true)
 |-- symptom_onset_date: string (nullable = true)
 |-- confirmed_date: date (nullable = true)
 |-- released_date: date (nullable = true)
 |-- deceased_date: date (nullable = true)
 |-- state: string (nullable = true)
 |-- no_days: integer (nullable = true)



### Remove null values of sex column.
### Add a is_male column if male then it should yield true, else (Female) then False

In [20]:
df = df.na.drop(subset=['sex'])

In [21]:
df = df.withColumn('is_male', F.when(df['sex'] == 'male', True).otherwise(False))

### Add a is_dead column if patient state is not released then it should yield true, else then False

- Use <b>UDF</b> to perform this task.
- However, UDF is not recommended there is no built in function can do the required operation.
- UDF is slower than built in functions.

In [23]:
# Define a UDF to determine if the patient is dead
def is_dead(state):
    return state != 'released'

is_dead_udf = udf(is_dead, BooleanType())



In [ ]:

df = df.withColumn('is_dead', is_dead_udf(df['state']))

### Change the ages to bins from 10s, 0s, 10s, 20s,.etc to 0,10, 20

In [24]:
df = df.withColumn('age', df['age'].substr(1, 2).cast('int'))

In [25]:
df.show(5)

+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+-------+-------+-------+
|patient_id|   sex|age|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|no_days|is_male|is_dead|
+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+-------+-------+-------+
|1000000001|  male| 50|  Korea|   Seoul| Gangseo-gu|     overseas inflow|       NULL|            75|        2020-01-22|    2020-01-23|   2020-02-05|   2020-02-05|released|     13|   true|  false|
|1000000002|  male| 30|  Korea|   Seoul|Jungnang-gu|     overseas inflow|       NULL|            31|              NULL|    2020-01-30|   2020-03-02|   2020-03-02|released|     32|   true|  false|
|1000000003|  male| 

### Change age, and no_days  to be typecasted as Double

In [26]:
df = df.withColumn('age', df['age'].cast('double'))
df = df.withColumn('no_days', df['no_days'].cast('double'))

### Drop the columns
["patient_id","sex","infected_by","contact_number","released_date","state",
"symptom_onset_date","confirmed_date","deceased_date","country","no_days",
"city","infection_case"]

In [27]:
columns_to_drop = ["patient_id", "sex", "infected_by", "contact_number", "released_date", "state",
                   "symptom_onset_date", "confirmed_date", "deceased_date", "country", "no_days",
                   "city", "infection_case"]
df_dropped = df.drop(*columns_to_drop)

In [28]:
df_dropped.show(5)

+----+--------+-------+-------+
| age|province|is_male|is_dead|
+----+--------+-------+-------+
|50.0|   Seoul|   true|  false|
|30.0|   Seoul|   true|  false|
|50.0|   Seoul|   true|  false|
|20.0|   Seoul|   true|  false|
|20.0|   Seoul|  false|  false|
+----+--------+-------+-------+
only showing top 5 rows



### Recount the number of nulls now

In [31]:
null_counts = df_dropped.select([F.sum(F.col(c).isNull().cast("int")).alias(c) for c in df_dropped.columns])
null_counts.show()

+---+--------+-------+-------+
|age|province|is_male|is_dead|
+---+--------+-------+-------+
|327|       0|      0|      0|
+---+--------+-------+-------+



## Now do the same but using SQL select statement

### From the original Patient DataFrame, Create a temporary view (table).

In [32]:
df.createOrReplaceTempView('view_patient_data')

### Use SELECT statement to select all columns from the dataframe and show the output.

In [34]:
spark.sql('SELECT * FROM view_patient_data').show(5)

+----------+------+----+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+-------+-------+-------+
|patient_id|   sex| age|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|no_days|is_male|is_dead|
+----------+------+----+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+-------+-------+-------+
|1000000001|  male|50.0|  Korea|   Seoul| Gangseo-gu|     overseas inflow|       NULL|            75|        2020-01-22|    2020-01-23|   2020-02-05|   2020-02-05|released|   13.0|   true|  false|
|1000000002|  male|30.0|  Korea|   Seoul|Jungnang-gu|     overseas inflow|       NULL|            31|              NULL|    2020-01-30|   2020-03-02|   2020-03-02|released|   32.0|   true|  false|
|1000000003|  m

### *Using SQL commands*, limit the output to only 5 rows

In [35]:
spark.sql('SELECT * FROM view_patient_data LIMIT 5').show()

+----------+------+----+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+-------+-------+-------+
|patient_id|   sex| age|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|no_days|is_male|is_dead|
+----------+------+----+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+-------+-------+-------+
|1000000001|  male|50.0|  Korea|   Seoul| Gangseo-gu|     overseas inflow|       NULL|            75|        2020-01-22|    2020-01-23|   2020-02-05|   2020-02-05|released|   13.0|   true|  false|
|1000000002|  male|30.0|  Korea|   Seoul|Jungnang-gu|     overseas inflow|       NULL|            31|              NULL|    2020-01-30|   2020-03-02|   2020-03-02|released|   32.0|   true|  false|
|1000000003|  m

### Select the count of males and females in the dataset

In [36]:
spark.sql('SELECT sex, COUNT(*) as count FROM view_patient_data GROUP BY sex').show()

+------+-----+
|   sex|count|
+------+-----+
|female| 2218|
|  male| 1825|
+------+-----+



### How many people did survive, and how many didn't?

In [37]:
spark.sql('SELECT state, COUNT(*) as count FROM view_patient_data GROUP BY state').show()

+--------+-----+
|   state|count|
+--------+-----+
|isolated| 1454|
|released| 2514|
|deceased|   75|
+--------+-----+



### Now, let's perform some preprocessing using SQL:
1. Convert *age* column to double after removing the 's' at the end -- *hint: check SUBSTRING method*
2. Select only the following columns: `['sex', 'age', 'province', 'state']`
3. Store the result of the query in a new dataframe

In [39]:
# Create a temporary view with age converted to double
spark.sql("""
    CREATE OR REPLACE TEMP VIEW view_patient_data_step1 AS
    SELECT
        sex,
        CAST(SUBSTRING(age, 1, LENGTH(age) - 1) AS DOUBLE) as age,
        province,
        state
    FROM view_patient_data
""")

DataFrame[]

In [40]:
# Select only the required columns and create a new temporary view
spark.sql("""
    CREATE OR REPLACE TEMP VIEW view_patient_data_step2 AS
    SELECT
        sex,
        age,
        province,
        state
    FROM view_patient_data_step1
""")

DataFrame[]

In [41]:
# Store the result in a new DataFrame
result_df = spark.sql("""
    SELECT
        sex,
        age,
        province,
        state
    FROM view_patient_data_step2
""")

# Show the result
result_df.show(5)

+------+----+--------+--------+
|   sex| age|province|   state|
+------+----+--------+--------+
|  male|50.0|   Seoul|released|
|  male|30.0|   Seoul|released|
|  male|50.0|   Seoul|released|
|  male|20.0|   Seoul|released|
|female|20.0|   Seoul|released|
+------+----+--------+--------+
only showing top 5 rows



## OR we can use it ...

In [38]:
result_df2= spark.sql("""
    SELECT
        sex,
        CAST(SUBSTRING(age, 1, LENGTH(age) - 1) AS DOUBLE) as age,
        province,
        state
    FROM view_patient_data
""")

# Show the result
result_df2.show(5)

+------+----+--------+--------+
|   sex| age|province|   state|
+------+----+--------+--------+
|  male|50.0|   Seoul|released|
|  male|30.0|   Seoul|released|
|  male|50.0|   Seoul|released|
|  male|20.0|   Seoul|released|
|female|20.0|   Seoul|released|
+------+----+--------+--------+
only showing top 5 rows



## Machine Learning
### Create a pipeline model to predict is_dead and evaluate the performance.
- Use <b>StringIndexer</b> to transform <b>string</b> data type to indices.
- Use <b>OneHotEncoder</b> to deal with categorical values.
- Use <b>Imputer</b> to fill missing data with mean.

In [64]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, Imputer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

In [53]:
selected_columns = ['age', 'sex', 'province', 'is_dead']
data = df.select(*selected_columns)

In [54]:
data.show(5)

+----+------+--------+-------+
| age|   sex|province|is_dead|
+----+------+--------+-------+
|50.0|  male|   Seoul|  false|
|30.0|  male|   Seoul|  false|
|50.0|  male|   Seoul|  false|
|20.0|  male|   Seoul|  false|
|20.0|female|   Seoul|  false|
+----+------+--------+-------+
only showing top 5 rows



In [55]:
data = data.dropna(subset=[col_name for col_name in selected_columns if col_name != 'is_dead'])


In [65]:
data = data.withColumn('is_dead', data['is_dead'].cast(IntegerType()))

In [66]:
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)


In [67]:
sex_indexer = StringIndexer(inputCol='sex', outputCol='sex_index')
province_indexer = StringIndexer(inputCol='province', outputCol='province_index')
sex_encoder = OneHotEncoder(inputCol='sex_index', outputCol='sex_vec')
province_encoder = OneHotEncoder(inputCol='province_index', outputCol='province_vec')

In [68]:
imputer = Imputer(inputCols=['age'], outputCols=['age'])
assembler = VectorAssembler(inputCols=['age', 'sex_vec', 'province_vec'], outputCol='features')

In [69]:
log_reg = LogisticRegression(featuresCol='features', labelCol='is_dead')

In [70]:
# Create pipeline
pipeline = Pipeline(stages=[sex_indexer, province_indexer, sex_encoder, province_encoder, imputer, assembler, log_reg])

In [71]:
# Fit the pipeline to training data
pipeline_model = pipeline.fit(train_data)


In [72]:
# Make predictions on test data
predictions = pipeline_model.transform(test_data)

In [73]:
# Evaluate performance
evaluator = BinaryClassificationEvaluator(labelCol='is_dead')
accuracy = evaluator.evaluate(predictions)

In [74]:
print("Accuracy:", accuracy)

Accuracy: 0.9048481431220015
